In [34]:
import warnings
import pandas as pd
import arviz as az

import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)
from libs.model import HGPforecaster
from libs.metrics import calculate_metrics, metrics_to_table
from libs.pre_processing import generate_groups_data
from libs.visual_analysis import visualize_fit, visualize_predict
az.style.use('arviz-darkgrid')
warnings.filterwarnings('ignore')

# Prison

In [35]:
prison = pd.read_csv('../data/prisonLF.csv', sep=",")
prison = prison.drop('Unnamed: 0', axis =1)
prison['Date'] = prison['t'].astype('datetime64[ns]')
prison = prison.drop('t', axis=1)
prison = prison.set_index('Date')
prison.columns= ['State', 'Gender', 'Legal', 'Count']

In [36]:
prison

,State,Gender,Legal,Count
Date,,,,
2005-03-01,ACT,Female,Remanded,2
2005-06-01,ACT,Female,Remanded,4
2005-09-01,ACT,Female,Remanded,1
2005-12-01,ACT,Female,Remanded,4
2006-03-01,ACT,Female,Remanded,4
...,...,...,...,...
2015-12-01,WA,Male,Sentenced,3894
2016-03-01,WA,Male,Sentenced,3876
2016-06-01,WA,Male,Sentenced,3969


In [37]:
prison.to_csv('../data/prison_to_r.csv')

# Tourism

In [38]:
data = pd.read_csv('../data/TourismData_v3.csv')
data['Year'] = data['Year'].fillna(method='ffill')

d = dict((v,k) for k,v in enumerate(calendar.month_name))
data.Month = data.Month.map(d)
data = data.assign(t=pd.to_datetime(data[['Year', 'Month']].assign(day=1))).set_index('t')
data = data.drop(['Year', 'Month'], axis=1)

In [39]:
data

,AAAHol,AAAVis,AAABus,AAAOth,AABHol,AABVis,AABBus,AABOth,ABAHol,ABAVis,...,GBBBus,GBBOth,GBCHol,GBCVis,GBCBus,GBCOth,GBDHol,GBDVis,GBDBus,GBDOth
t,,,,,,,,,,,,,,,,,,,,,
1998-01-01,2015.444457,1379.273926,230.099578,118.238146,936.208816,297.944688,0.000000,0.000000,915.526615,361.161122,...,0.000000,0.000000,7.536223,0.000000,1.628948,0.000000,0.811856,0.000000,9.478051,0.0
1998-02-01,514.337600,645.980857,450.509894,80.836085,139.829461,96.072530,3.800646,5.862538,211.001057,346.151398,...,1.045797,0.000000,0.000000,0.000000,5.296459,0.000000,0.522899,0.000000,0.000000,0.0
1998-03-01,532.097470,683.284592,520.458259,138.306554,117.980175,97.113121,28.431680,0.000000,227.458798,185.651450,...,0.000000,0.000000,2.945006,1.425324,9.924744,3.100121,0.000000,0.000000,0.000000,0.0
1998-04-01,534.059083,819.807646,755.506898,97.089844,202.266724,196.414355,10.812915,0.000000,1273.068333,265.980973,...,11.461824,0.000000,26.419177,13.690603,2.312088,0.000000,0.000000,10.958005,2.312088,0.0
1998-05-01,505.222272,693.531752,880.152314,82.210783,121.775526,73.542768,12.185418,24.524678,259.541675,169.085846,...,0.000000,0.000000,23.789282,67.846207,1.282767,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-08-01,383.526216,838.269610,455.637579,458.064241,86.281072,83.870793,0.000000,0.000000,228.048826,243.160300,...,7.874448,0.000000,49.899947,8.300225,17.633477,8.218562,4.121631,0.408894,1.625820,0.0
2016-09-01,484.959999,512.190298,451.742958,194.214472,120.465489,46.129082,251.544316,0.000000,181.893020,333.526684,...,0.421996,0.000000,80.058887,37.306013,109.024164,46.447153,11.661140,3.557735,4.448534,0.0
2016-10-01,399.867668,907.197296,527.255353,102.575984,201.400342,118.033070,6.457149,4.387921,444.169346,229.721638,...,35.375093,0.273247,52.156131,2.093902,50.283538,3.319366,0.754941,0.000000,0.000000,0.0
